# 1 - Randomised Controlled Trial (RCT)

> 👉 해당 포스트는 아래 인과추론 자료를 바탕으로 정리했습니다.
> - [고수들의 계량경제학 1장 무작위 시행](https://product.kyobobook.co.kr/detail/S000001059425)
> - [Causal Inference for the brave and true Chapter 2. Randomised Experiments](https://matheusfacure.github.io/python-causality-handbook/02-Randomised-Experiments.html) / [한국어 번역 자료](https://causalinferencelab.github.io/Causal-Inference-with-Python/02-Randomised-Experiments.html)
> - [인과추론의 데이터 과학](https://sites.google.com/view/causal-inference2022)

## 0. Chapter Summary

> - 배운내용을 어떻게 `현업에서 활용할지`  & `사례를 바탕으로 느낀점`에 중점
> - 단순 책 내용을 요약하는 건 앞으로 실제 업무 활용 시, 도움이 안될 수 있으므로 `본인만의 방식으로 재구조화`

<aside>

1. `목적` : 해당 챕터를 배우는 이유는 무엇일까? 
    - 실험 디자인 시, 이상적이지만, Treatment 인과효과 추정에 가장 효과적인 방법

<aside>

2. `내용` : 이번 챕터에서 어떤 메세지를 전달하려고 하는가?
    - 모집단을 가지고 실험을 하는 것은 현실적으로 어려움. 따라서, 모집단과 유사한 그룹 샘플 데이터를 바탕으로 **Random Assignment** 하는 것이 핵심
    - Randomised Experiments 상황에서, **Association** = **Causation**
        - 결국, 유저를 랜덤하게 나눌 수 있는 환경이면, 복잡한 실험디자인 or 통계 모형을 사용하지 않아도 **Causal Effect 추정**이 가능
    - RCT 이후에 배울 내용 IV, DID, RDD, Matching, Synthetic Control이 왜 필요한지 내포하는 챕터 
        - 참고로, 위의 방법론은 **실험 디자인**을 잘 활용해서, 두 그룹을 비교 가능하게 만들어주는 방법

<aside>

3. `느낀점` : 배운 내용을 어떻게 현업에서 적용해볼 수 있는가? 왜 적용이 어려울까?
    - 직관과 실험 : 어떻게 보면 당연한, 검증 작업에 직관보다는 실험이 필요하지만, 아래 상황들로 인해 여의치 않음
    - 조직 문화 : OCE (Online Controlled Experiment) 플랫폼을 구축하는 것은 혼자 할 수 없으며, 조직 차원에서 실험의 중요성을 이해하고 있어야 함
        - NAVER Search A/B Test 플랫폼 [[Blog Link]](https://brunch.co.kr/@lifidea/42) / [[Video]](https://tv.naver.com/v/23651289)
        - 오늘의 집 A/B 실험 플랫폼 구축기 [[Link]](https://www.bucketplace.com/post/2021-10-29-%EC%98%A4%EB%8A%98%EC%9D%98%EC%A7%91-a-b-%EC%8B%A4%ED%97%98-%ED%94%8C%EB%9E%AB%ED%8F%BC-%EA%B5%AC%EC%B6%95%EA%B8%B0/)
    - 리소스 : 이해 부서와 실험 설계를 같이하고 분석을 진행하는 것은 시간과 비용 소모가 큰 작업. 따라서, 실험 설계단계에서 각 조직의 성과 지표 Align이 되었는지 확인 필요
    - 유저 경험 : 동일 빌드에서 어떤 유저에게만 프로모션을 하는 것은 라이브 서비스 측면에서 안좋을 수 있음
    - 분석가의 고충
        - 기획/개발 단계에서 검증해야할 지표가 제대로 협의/논의되지 않음
        - Data Generating Process : 로그는 잘 남고 있을까?
        - 실험을 진행할 유저군이 랜덤하게 나뉘어지지 않음 (**Selection Bias**)
        - **Confounding Factor** 제대로 통제하지 못하는 케이스가 존재 (기간 / 그룹특성)

<aside>

4. `어려운점` : 어떤 부분이 해당 챕터를 다룰 때 어렵고 생소했나?
    - 도메인 지식 : 미국의 의료보험 체계 및 사회적인 배경을 잘 몰라서, 내용을 이해하는데 까다로웠음
    - 가설검정과 인과추론 : 어떤 부분에서 관련이 있는지 생각하는데, 시간을 많이 쏟음

</aside>


## 1. Introduction to Causal Inference

<aside>

> 💡Causal Inference is concerned with a **very specific kind of prediction problem**
> : Predicting the results of an action, manipulation, or Intervention
>  - “Making Things Happen” (2003, Woodward)

</aside>

- 인과추론 (Causal Inference)이란 무엇일까?
    - 문제에 대한 원인을 찾고 해당 원인에 대한 효과를 추론하는 것
    - 즉, **Treatment**를 주었을 때, 이에 따른 **Outcome**이 어떻게 바뀌는지를 추정
        - **이벤트**(Treatment)를 진행하면, 유저의 **잔존율**(Outcome)이 높아질까?

<aside>

- 그러면 ML과 무엇이 다른가? `< Causal Inference vs Machine Learning >`
    - Causal Inference : **Potential outcomes**까지 고려
    - Machine Learning : **Observed outcomes**만을 고려

<aside>

- 상관관계는 인과관계를 의미하지 않음. Why? ***Confounding Factors***
    
    ![img](./Data/5th_PseudoCon_Confounders.png)
    
    <aside>
    
    ✅  **Standard Approach** & **Causal Approach**
    
    - **기본 접근** : $X$의 변화가 $Y$의 변화와 어떻게 **연관**되어 있는지 정량화하는데 관심
        - $Y = β_0 + β_1X+ ε$ → $E(Y|X=x+1) - E(Y|X=x)$
        - Regression (Chapter 2)에서 다룰 기본적인 접근
    - **인과추론 접근** : $X$ (원인)의 변화가 $Y$의 변화를 **유발**하는지를 확인하는데 관심
        - 해당 접근은 **Y가 변하는 이유(원인)**에 대한 질문에 답을 할 수 있음
        - 만약 $X$와 $Y$가 인과적으로 관계가 있다면, $Y$의 변화는 $X$의 변화로 설명 가능
    </aside>
    

## 2. Experiment Key Takeaway

<aside>

> ▶️ 건강보험이 건강에 미치는 인과효과 추정 <[고수들의 계량경제학 Chapter 1. 무작위 시행](https://product.kyobobook.co.kr/detail/S000001059425)>

</aside>

### 1) 실험 목적

- 목적 : 보험 가입이 실험 대상에 미치는 영향을 2가지 관점에서 파악 
→ 실험 규모와 비용을 고려 시, 여러 가지의 목표 지표 설정이 가능하도록 설계 할 수 있음
    - 가설 1 : 보험 의료 가격이 감소하면, 실제로 의료 서비스를 더 사용할 것이다. → Yes
    - 가설 2 : 보험 가입을 통해, 건강 증진의 인과적 효과가 있을 것이다. → No

<aside>

### 2) 실험 설계

- 배경 : 국가 정책을 위해, 영향을 받는 모든 국민을 대상으로 테스트하면 아래와 같은 문제 발생 할 수 있음
    - 국가적으로 너무나 큰 리소스가 들어가며 통제 하기 쉽지 않음
    - 또한, Random Assignment 과정에서 윤리적인 부분이 이슈가 될 수 있음
</aside>

- 목적 : 모집단 (Population)을 잘 반영하는 실험 유저군 (Sampling Group)을 적절히 샘플링
    - Sampling Bias 최소화
    - Apple to Apple (Random Assignment)

</aside>

- 실험 그룹 설계 : 올바른 실험설계를 통해, 목적에 맞는 인과효과를 추정하기 위함
    1. `테스트 가능한 가설 설계 및 목표 지표 설계` 
        - **Primary Index** <궁극적인 목표가 되는 지표> : 지출한 의료비 / 건강 지표
        - **Secondary Index** <실험과 직접적 연관이 되는 지표> : 부담 보험료
        - **Guardrail Index** <실험 과정에 부정적 영향을 받을 수 있는 지표> : 건강 지표 [푸시를 했는데 이탈하는 유저가 발생한 Nexon 사례](https://www.intelligencelabs.tech/5e047ef8-f811-47a8-9058-e6dfb2d5f8aa)

    <aside>

    2. `실험군/대조군 설정` : 보험의 보장 수준 (가입자 부담 보험료 수준)에 따라 5개 상품으로 구분 
        - **Control Group** : 무보험 상태에 가까운 보험 수준을 가진 상품 (재난적 플랜)
        - **Treatment Group** : 그 외 보험 보장이 되는 상품군 (4가지)

    <aside>
        
    3. `방법` : 보험 미가입자를 대상으로 5개의 보험 상품에 **Random Assignment**

    <aside>

    4. `그룹 검증` : 과연 랜덤하게 나눈 실험군과 대조군이 서로 비교 가능한가 (**Ceteris Paribus**)
        - 실험 대상 (유저/국민)을 대표할 수 있는 비교 변수 설정 (Ex. 나이/연령/과금 수준 등)
        - 통계적으로 유의미한 차이가 발생했는지 확인 → 상황에 따라 [A/A 테스트 진행](https://yozm.wishket.com/magazine/detail/1257/)을 하기도 함
        - 표본의 수가 충분한지 (Law of Large Numbers 나온 배경) & 동일한 수준으로 제대로 나뉘었는지 체크 필요
        
        → 해당 실험에서 그룹간 차이는 무시해도 되는 것으로 보여짐
        

### 3) 실험 진행 & 분석

- 실험 진행 시 체크 사항
    - 지표 모니터링 : 실험이 진행되는 동안, 실험 대상/유저가 받게될 경험에 부정적인 요소가 있는지 & 실험에 영향을 주는 외부 요인이 있는지 모니터링
    - 로그 확인 : 실험 분석에 진행될 로그가 잘 쌓이고 있는지 확인

<aside>

- 실험 분석 : 리포트 및 실험 Dashboard 제공
    - 해당 실험을 통해, 유저의 경험을 어떤 측면에서 개선했는지 사전 설계 지표 및 실험 그룹을 바탕으로 성과 분석

<aside>

### 4) 실험 의사결정 & Feedback

- 목적 :
    - 실험을 바탕으로 조직 내 의사결정에 활용 → 보험 가입이 건강 증진에 도움이 될까? No
    - 이번 실험에서 얻은 Insight와 보완점을 통해, 이후 실험 과정 개선
    - (반복 실험이 가능하다면) 이를 바탕으로, 실험을 개선 결과의 신뢰성을 높이기 위해 노력

<aside>

- 건강 보험 실험 Feedback
    - 실험 설계의 문제 : 건강보험이라는 국가적 실험에서 완벽한 **Random Assignment**가 된 것이 맞을까?
        - `Sampling Bias` : 모집단 (실제로 보험에 가입하지 않은 사람들)과 샘플 그룹 (재난적 플랜에 가입된 Control Group) 간의 차이가 존재
        - `Unobserved Confounders` : 관측된 인구통계 정보를 바탕으로 설계를 했을 때, 약간의 Sampling Bias 존재. 그렇다면, 관측되지 않은 변수에서는 해당 부분이 더 크게 나타날 수 있지 않을까?
        - `Censoring Issue` : 실험 중도 이탈자 / 실험군이지만, 영향을 받지 않은 유저는 어떻게 처리를 했는가? 제외했다면 Selection Bias가 아닌가?

    <aside>

    - 결과의 유효성 : 보험 증진으로 건강 개선을 할 수 없었던 것이 맞을까?
        - `목표 지표` : 건강 지표는 과연 객관적으로 정량화가 가능한 부분인가?
        - `실험 개선` : 보험 가입에 의한 효과가 없다면 이후의 실험 개선은? **오리건의 건강 보험 실험**

## 3. Theoretical Backgroud

### 1) 실험 설계와 방법론 이해의 필요성

- 목적 : 결국, 인과추론의 근본적인 문제를 이해하고 효과적으로 해결해나가기 위함
    
    > `Treatment` : 접속 이벤트 참여 여부
    > 
    > - 실험군 (Treatment Group) : 이벤트 참여에 배정된 유저군
    > - 대조군 (Control Group) : 이벤트 참여에 배정되지 않은 유저군
    > 
    > `Outcome` : 유저 잔존율
    > 
    > - 비교 그룹간 유의미한 차이 (ATE, Average Treatment Effect)가 있는지 확인하는 지표

<aside>

- 문제 : **Counterfactuals** (Counter to fact, 일어나지 않은 상황을 가정)
    - 실험 진행 시, 유저는 참여/미참여 중 한 개의 상태로만 존재할 수 있음
    - 따라서, 실험 당시 미참여 유저가 그렇지 참여한 상황(실제로 일어나지 않음)을 가정
    - 하지만, 우리는 타임머신이 없기 때문에 동일한 유저에 대해서 2가지 사항을 관측 불가
        
        → **인과추론의 근본적인 문제** <Fundamental Problem of Causal Inference>
        
        ![img](./Data/5th_PseudoCon_Counterfactuals.png)
        

### 2) 인과추론의 근본적인 문제 접근하기

- 하나의 실험 대상에 대해 Treatment에 대한 Potential Outcomes 모두 관찰 불가
    - **Selection Bias** 발생하는 이유?
        - 인과추론 근본적인 문제 : `Control Group ≠ Counterfactuals`
        - 특성이 다른 다른 대상과 비교 시 **Selection Bias**가 발생 : `<쿠즈다르와 마리아 사례>`
    - 결국, 이 문제가 해결이 되어야 Treatment에 따른 인과적인 효과 파악이 가능

<aside>

- `Individual Treatment Effect (ITE)` : 개별 유저 *i* 에 대해 Treatment 처지 효과
    - 유저 *i* 에게는 2가지 Potential Outcomes이 존재
        - $T = 1$ : 이벤트 참여 / $T = 0$ : 이벤트 미참여
        - 2가지 Potential Outcomes 중에서 하나만 존재할 수 있음
        - 유저 *i* 에 대한 개별 인과효과 (ITE)
            
            $
            ITE = Y_{0i} - Y_{1i}
            $
            
- `Average Treatment Effect (ATE)` : 유저 그룹에 대한 Treatment 처지 효과
    - **유저 개인화 관점**에서는 ITE가 이상적이고 중요하지만, 대부분은 유저 그룹단위의 실험이 일반적이며 개개인에 대해 ITE를 파악할 수 없는 경우가 존재
    - 따라서, 유저 개인의 인과효과를 평균을 내어 집단 레벨에서 설명
    - 만약, 제대로 된 실험 설계를 하지 않고 ATE를 계산한다면? 아래와 같은 Selection Bias가 생김
    <[Causal Inference for the brave and true Chapter1. 수식 참조](https://matheusfacure.github.io/python-causality-handbook/01-Introduction-To-Causality.html)>
        
    $
    E[Y|T=1] - E[Y|T=0] = E[Y_1|T=1] - E[Y_0|T=0] + E[Y_0|T=1] - E[Y_0|T=1]
    $

    $
    E[Y|T=1] - E[Y|T=0] = \underbrace{E[Y_1 - Y_0|T=1]}_{ATT} + \underbrace{\{ E[Y_0|T=1] - E[Y_0|T=0] \}}_{BIAS}
    $
        

### 3) How to solve?

- 인과추론의 근본적인 문제를 바라보는 3가지 관점
    - `Potential Outcomes` : 물음표 채우기
    - `Structural Causal Models` : DAG
    - `Regression` → 다음 챕터가 Regression인 이유!
        - 오차항 가정(Gaussian Assumption)과 내생성 (Endogenity) → 도구변수 참조

<aside>    

- **Selection Bias 해결**하기 위해 이번 챕터에서는 **Random Assignment** 도입
    - 즉, 실험 대상를 동전던지기로 나눠서 Treatment 여부를 결정

<aside>

- Random Assignment이 가장 좋은 방법이지만, **Research Design** 필요성 존재
    - 목적 :
        - 하지만, 항상 주어진 상황에서 RCT를 활용할 수 없는 경우가 많음
        - 위와 같은 경우, `Counterfactual과 최대한 비슷한 Control Group`를 실험 디자인을 통해 찾아나가야 함 ↔ `Selection Bias 줄이기`

    <aside>

    - 실험 디자인 방법론 (To be continued)
        - `Instrumental Varibles` (2SLS / Regression, Chapter 3)
        - `RDD` (Regression Discontinuity Design, Chapter 4)
        - `DID` (Difference In Difference, Chapter 5)
        - `Synthetic Control`

### 4) Statistical Theory

- LLN (Law of Large Numbers) 나오게 된 배경
    - 결국 모집단을 잘 대표하는 표본(Sample)을 선정하기 위해, 충분한 수의 표본이 필요
    - 궁극적으로, 실험군과 대조군은 동일한 모집단에서 생성 → 그룹간에 비교 가능한 특성을 가져야함 → 해당 조건 달성을 위해서는 충분한 표본이 필요
    - 과연 Sample Size는 어떤 수준이 적절할까? 
        - Statistical Power(검정력)과 Sample Size
        - Product Active User 규모를 고려

<aside>

- Hypothesis Testing
    - 목적 : 모집단 (Population Data)의 특성에 대해 설계한 통계적 가설 ($H_0$ / $H_1$)을 모집단의 추출한 샘플 데이터 (Sample Data)를 이용해 검증하는 과정
    - 인과추론과 가설 검정 : 결국, 인과적인 효과를 추정하기 위해 모집단의 데이터를 활용하는 것보다는 샘플 데이터를 이용해, 개입 (Intervention)에 따른 효과가 있는지 검증
        - 그래서, 후반부에 가설 검정 T-test ↔ Two Sample T-test (↔ Hausman Test)이 나오게된 것 같음




## 3. A/B 테스트 Open Source

- A/B 테스트를 할 수 있는 오픈 소스 및 자료
    - 자체 OCE (Online Controlled Experiment) 플랫폼이 있다면 최적의 환경
    - 실제로 3rd Party 툴 (Amplitude / Braze / Firebase 등)을 통해서 A/B Test를 해볼 수 있음
    - [GrowthBook](https://www.growthbook.io/)과 같이 잘 알려진 오픈 소스를 통해, 현업에 적용이 가능
    - 오픈 소스 정리자료 링크 : https://posthog.com/blog/best-open-source-ab-testing-tools

<aside>

- 실험 플랫폼 
    - 직관이 아닌 실험으로의 의사결정은 조직에서 매우 중요한 과제
    - 참고 도서 : [실리콘밸리의 실험](https://product.kyobobook.co.kr/detail/S000200763627)
    

## 😍 Q & A

- (종언) 실험에 사용할 샘플 사이즈는 어떻게 가져가는게 좋을까요?
    - (이삭) : 통계학적인 방법으로는 Statistical Power과 샘플 사이즈가 양의 상관관계가 있어, 검정력을 기준으로 샘플 사이즈가 적절한지 판단하는 것 같습니다,
    - (소희) : 적정 샘플 사이즈를 계산해주는 [사이트](https://www.calculator.net/sample-size-calculator.html)를 사용해봤어요.
    - (진수) : 오히려 너무 적은 극단적인 케이스는 Bias가 많고 일반적으로 정상적인 범주에서 벗어났다고 생각해, 실험 대상으로는 제외했던 기억이 있습니다. 원론적인 답변이지만 항상 Product의 Active User 수를 고려해야 할 것 같습니다.

<aside>

- (이삭) 의료 보험 가입으로 건강 증진의 효과를 얻지 못했는데, 미국의 사례여서 그런걸까요?
    - (정현) : 의료 보험 체계가 다르지만 한국도 비슷하지 않을까 싶습니다. 그런데 실험을 해보지 않아서 직관적인 판단에 주의해야할 것 같아요.
    

## 😍 Reference

- [고수들의 계량경제학 Chapter 1. 무작위 시행](https://product.kyobobook.co.kr/detail/S000001059425) <Joshua D. Angrist , Jorn-Steffen Pischke 저>
- [Causal Inference for the brave and true Chapter 1. Introduction to Causality](https://causalinferencelab.github.io/Causal-Inference-with-Python/01-Introduction-To-Causality.html) <Matheus Facure 저>
- [Korea Summer Workshop on Causal Inference 2022](https://sites.google.com/view/causal-inference2022)
- [Introduction to Causal Inference](https://www.bradyneal.com/causal-inference-course)
- [유 퀴즈 온 더 블럭 - 구준엽편](https://www.youtube.com/watch?v=CYfbQ3_dR0U)